<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/%EA%B0%80%EC%98%81%EC%95%84%EC%9D%B4%EA%B1%B0%EC%95%BC(law_%EC%B6%94%EA%B0%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 

In [3]:
! pip install -q openai langchain tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 14.7 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage

In [6]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

In [7]:

upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="embedding-passage")

In [8]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


# Cornell.edu
**알파벳 전체를 돌면서 URL 리스트 만들기**

In [9]:
import time
import string
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

In [10]:
base = "https://www.law.cornell.edu"
index_template = base + "/wex/all/{letter}"

header = {"User-Agent": "law/0.1 (for project; contact:seoyen1122@gmail.com)"}

In [11]:
def collect_links(letters=None):
  if letters is None:
    letters = list(string.ascii_lowercase)

  seen = set()
  entries = []

  for letter in letters:
    index_url = index_template.format(letter = letter)
    resp = requests.get(index_url, headers = header) #requests- 웹사이트에 요청보내고응답받기, index_url 주소로 httpget 요청 보냄, header = 나는 이런 이름의 봇이고 프로젝트 목적으로 요청 보낸다는 정보 전달용, return: request.Response객체, 이 페이지 정보, resp.status_code, resp.text, resp.content
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser") #다운받은 html문자열을 파싱해서 태그들을 쉽게 찾을 수 있는 객체로 바꾸는 코드.

    for a in soup.find_all("a", href=True): #이 페이지 안에서 href 속성이 있는 모든 <a> 태그를 찾아라 → 그 안에 들어있는 링크 주소(href)를 하나씩 꺼내자
      href = a["href"]

      if not href.startswith("/wex/"):
        continue
      if "/wex/all" in href:
        continue

      full_url = urljoin(base, href)
      if full_url in seen:
        continue

      term = a.get_text(strip=True)
      entries.append({"term": term, "url":full_url})
      seen.add(full_url)

    time.sleep(3)

  print("total entries:", len(entries))
  return entries

**각 항목 페이지에서 본문 텍스트 추출**

상단 네비게이션

h1 제목(용어)

그 아래 여러 단락/리스트가 정의/설명

맨 아래에 wex toolbox, terms of use같은 공통 푸터

main 안에 있는 p, ul, ol 태그 텍스트를 합치는 식으로 본문만 자르기

In [12]:
def fetch_article(url):
  resp = requests.get(url, headers = header)
  resp.raise_for_status()
  soup = BeautifulSoup(resp.text, "html.parser")

  main = soup.find("main") or soup

  #제목(h1/h2)
  title_tag = main.find(["h1", "h2"])
  title = title_tag.get_text(strip=True) if title_tag else ""

  #푸터/툴박스 부분 대충 제거 (string 에 "Wex Toolbox"가 들어가면 그 뒤는 날려버기)
  toolbox = main.find(string=lambda s: isinstance(s, str) and "Wex Toolbox" in s)
  if toolbox:
    parent = toolbox.parent
    sib = parent.next_sibling
    while sib is not None:
      next_sib = sib.next_sibling
      try:
        sib.decompose()
      except Exception:
        pass
      sib = next_sib

  chunks = []
  current_title = title or "Definition"
  current_texts = []

  for element in main.find_all(["h2", "p", "li"], recursive=True):
    if element.name == "h2":
      if current_texts:
        text = "\n\n".join(current_texts).strip()
        if text:
          chunks.append({
              "section_title": current_title,
              "text": text
          })
        current_texts = []
      current_title = element.get_text(" ", strip=True) or title

    elif element.name in ["p", "li"]:
      txt = element.get_text(" ", strip=True)
      if txt:
        current_texts.append(txt)

  if current_texts:
    text = "\n\n".join(current_texts).strip()
    if text:
      chunks.append({
          "section_title": current_title,
          "text": text
      })


  # 혹시 아무 청크도 못 만들었으면, 전체 p/li를 한 덩어리로라도 넣기 (optional)
  if not chunks:
      texts = []
      for element in main.find_all(["p", "li"]):
          txt = element.get_text(" ", strip=True)
          if txt:
              texts.append(txt)
      if texts:
          chunks.append({
              "section_title": title or "Definition",
              "text": "\n\n".join(texts)
          })

    # 최종 메타데이터 dict 반환
  meta = {
      "source_url": url,
      "title": title,
      "doc_type": "wex",
      "chunk_list": chunks,
  }
  return meta

**전체 크롤링 + jsonl로 저장**



In [13]:
import json

In [14]:
def crawl(output_path="wex_raw.jsonl", letters=None):
  entries = collect_links(letters)

  with open(output_path, "w", encoding="utf-8") as f:
    for i, entry in enumerate(entries):
      url = entry["url"]
      meta = fetch_article(url)

      if meta.get("chunk_list"):
        f.write(json.dumps(meta, ensure_ascii=False) + "\n")
        f.flush()

      time.sleep(3)


# 판례

In [15]:
case_index_url = base + "/supct/cases/name.htm"

In [16]:
def collect_case_links():

    resp = requests.get(case_index_url, headers=header)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    entries = []
    seen = set()

    # 이 페이지에는 A~Z 리스트 안에 <a>들이 잔뜩 있음
    for a in soup.find_all("a", href=True):
        href = a["href"]

        # Cornell historic decisions URL 패턴들 (두 가지 정도 있음)
        if not (href.startswith("/supct/html/") or href.startswith("/supremecourt/text/")):
            continue

        full_url = urljoin(base, href)
        if full_url in seen:
            continue

        title = a.get_text(strip=True)
        entries.append({"title": title, "url": full_url})
        seen.add(full_url)

    print("total historic SCOTUS cases:", len(entries))
    return entries

**판례case fetch**

In [17]:
def fetch_case_article(url):
    """
    판례 페이지에서:
    - title (사건명)
    - Facts / Issue / Holding / Reasoning 등 섹션을 찾아서
    Wex와 동일한 스키마로 meta를 반환.
    """
    resp = requests.get(url, headers=header)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # 사이트에 맞게 main 영역 / content 영역 선택
    main = soup.find("main") or soup

    # 제목
    title_tag = main.find(["h1", "h2"])
    title = title_tag.get_text(strip=True) if title_tag else ""

    chunks = []

    # (예시) 섹션 제목이 h2/h3 로 되어 있다고 가정
    current_title = "Facts"
    current_texts = []

    for element in main.find_all(["h2", "h3", "p", "li"], recursive=True):
        if element.name in ["h2", "h3"]:
            # 새 섹션 시작 → 이전 섹션 flush
            if current_texts:
                text = "\n\n".join(current_texts).strip()
                if text:
                    chunks.append({
                        "section_title": current_title,
                        "text": text,
                    })
                current_texts = []
            current_title = element.get_text(" ", strip=True) or current_title

        elif element.name in ["p", "li"]:
            txt = element.get_text(" ", strip=True)
            if txt:
                current_texts.append(txt)

    if current_texts:
        text = "\n\n".join(current_texts).strip()
        if text:
            chunks.append({
                "section_title": current_title,
                "text": text,
            })

    # fallback: 그래도 없으면 전체 p/li 한번에
    if not chunks:
        texts = []
        for element in main.find_all(["p", "li"]):
            txt = element.get_text(" ", strip=True)
            if txt:
                texts.append(txt)
        if texts:
            chunks.append({
                "section_title": title or "Opinion",
                "text": "\n\n".join(texts),
            })

    meta = {
        "source_url": url,
        "title": title,
        "doc_type": "case",
        "chunk_list": chunks,
    }
    return meta

**판례case 크롤링**

In [18]:
def crawl_cases(output_path="cases_structured_all.jsonl", case_urls=None):
    """
    case_urls: 미리 수집해둔 판례 URL 리스트 (LII/Oyez 등)
    """
    if case_urls is None:
        entries = collect_case_links()
        case_urls = [e["url"] for e in entries]

    with open(output_path, "w", encoding="utf-8") as f:
        for i, url in enumerate(case_urls):
            try:
                meta = fetch_case_article(url)
            except Exception as e:
                print(f"[WARN] failed to fetch {url}: {e}")
                continue

            if meta.get("chunk_list"):
                f.write(json.dumps(meta, ensure_ascii=False) + "\n")
                f.flush()

            time.sleep(3)

    print("cases saved to", output_path)



**청킹 + 임베딩 + FAISS**

In [19]:
import os
os.environ["UPSTAGE_API_KEY"] = UPSTAGE_API_KEY

In [20]:
import json
import faiss
import numpy as np
from langchain_upstage import UpstageEmbeddings

In [21]:
def chunk_text(text, max_chars=800, overlap_chars=200):
  chunks = []
  start = 0
  n = len(text)
  while start < n:
    end = min(start + max_chars, n)
    chunk = text[start:end]
    chunks.append(chunk)
    if end == n:
      break
    start = end - overlap_chars
  return chunks

def load_chunks_multi(jsonl_paths, max_chars=None, overlap_chars=200):
    chunks = []
    metadata = []

    for jsonl_path in jsonl_paths:
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                rec = json.loads(line)
                source_url = rec["source_url"]
                title = rec["title"]
                doc_type = rec.get("doc_type", "wex")  # ★ Wex vs case 구분
                chunk_list = rec.get("chunk_list", [])

                for sec_idx, c in enumerate(chunk_list):
                    section_title = c.get("section_title", title)
                    text = c.get("text", "")
                    if not text.strip():
                        continue

                    if max_chars is None:
                        chunks.append(text)
                        metadata.append({
                            "source_url": source_url,
                            "title": title,
                            "doc_type": doc_type,
                            "section_title": section_title,
                            "section_index": sec_idx,
                            "subchunk_index": 0,
                        })
                    else:
                        for sub_idx, ch in enumerate(
                            chunk_text(text, max_chars, overlap_chars)
                        ):
                            chunks.append(ch)
                            metadata.append({
                                "source_url": source_url,
                                "title": title,
                                "doc_type": doc_type,
                                "section_title": section_title,
                                "section_index": sec_idx,
                                "subchunk_index": sub_idx,
                            })

    return chunks, metadata

In [22]:
def build_faiss_index(
    jsonl_paths = ("wex_structured_all.jsonl", "cases_structured_all.jsonl"),
    index_path = "law_faiss.index",
    meta_path = "law_metadata.jsonl",
):
    chunks, metadata = load_chunks_multi(jsonl_paths, max_chars=800, overlap_chars=200)
    print("num_chunks:", len(chunks))

    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

    vecs = embeddings.embed_documents(chunks)
    emb = np.array(vecs, dtype=np.float32)
    print("emb shape", emb.shape)

    dim = emb.shape[1]
    faiss.normalize_L2(emb)
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    faiss.write_index(index, index_path)

    with open(meta_path, "w", encoding="utf-8") as f:
        for m in metadata:
            f.write(json.dumps(m, ensure_ascii=False) + "\n")

    print("index saved to", index_path)
    print("metadata saved to", meta_path)

In [23]:
# 1) Wex 크롤링 → wex_structured_all.jsonl 생성
crawl(output_path="wex_structured_all.jsonl", letters=None)

# 2) 판례 크롤링 → cases_structured_all.jsonl 생성
crawl_cases(output_path="cases_structured_all.jsonl")

KeyboardInterrupt: 

In [ ]:
# 3) 두 JSONL을 함께 써서 Law FAISS 인덱스 생성
build_faiss_index(
    jsonl_paths=("wex_structured_all.jsonl", "cases_structured_all.jsonl"),
    index_path="law_faiss.index",
    meta_path="law_metadata.jsonl",
)

num_chunks: 105996


In [ ]:
from langchain_upstage import UpstageEmbeddings
import numpy as np
import faiss
import json

def build_faiss_index(
    jsonl_path=["wex_structured_all.jsonl", "cases_structured_all.jsonl"],
    index_path="law_faiss.index",
    meta_path="law_metadata.jsonl",
):
    # 1) 청크 + 메타데이터 로드
    chunks, metadata = load_chunks_multi(jsonl_path, max_chars=800)
    print("num_chunks:", len(chunks))

    # 2) Upstage 임베딩 모델 (★ suffix 빼고 base 이름만 사용)
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

    # 3) 배치로 나눠서 embed_documents 호출 (예: 256개씩)
    all_vecs = []
    batch_size = 256

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i : i + batch_size]
        # 혹시라도 공백 문자열이 섞여 있으면 제거 (안전)
        batch = [c for c in batch if c.strip()]
        if not batch:
            continue

        vecs = embeddings.embed_documents(batch)
        all_vecs.extend(vecs)
        print(f"embedded {i + len(batch)} / {len(chunks)}")

    emb = np.array(all_vecs, dtype=np.float32)
    print("emb shape", emb.shape)

    dim = emb.shape[1]
    faiss.normalize_L2(emb)
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    faiss.write_index(index, index_path)

    with open(meta_path, "w", encoding="utf-8") as f:
        for m in metadata:
            f.write(json.dumps(m, ensure_ascii=False) + "\n")

    print("index saved to", index_path)
    print("metadata saved to", meta_path)


In [ ]:
build_faiss_index(
    jsonl_path="wex_structured_all.jsonl",
    index_path="wex_faiss_all.index",
    meta_path="wex_metadata_all.jsonl",
)

In [ ]:
pip install jsonlines

In [ ]:
#!/usr/bin/env python3
"""
create_faiss_index.py

- 'entries.jsonl' (Semantic Chunking된 파일)을 읽어옵니다.
- 각 청크(섹션)를 로드합니다.
- (안전 장치) 만약 섹션 텍스트가 1000자를 넘으면, 1000자 단위로 더 잘게 자릅니다.
- 이 과정에서 'title', 'source_url', 'section_title' 메타데이터를 모두 보존합니다.
- BGE-small 임베딩 모델을 사용하여 모든 청크를 임베딩합니다.
- 'faiss_index_philosophy'라는 이름의 로컬 FAISS 인덱스로 저장합니다.
"""

import jsonlines
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import sys
import time

# ----------------------------------------------------
# 1. 설정값
# ----------------------------------------------------
JSONL_FILE = ["wex_structured_all.jsonl", "cases_structured_all.jsonl"]           # 입력 파일 (스크래핑 결과)
INDEX_NAME = "faiss_index_law"    # 저장할 FAISS 인덱스 이름

# "Safety Net" 청킹 설정 (H2 섹션이 너무 클 경우 대비)
CHUNK_SIZE = 1000   # 청크 최대 글자 수
CHUNK_OVERLAP = 100 # 청크 겹침

# ----------------------------------------------------
# 2. 임베딩 모델 로드
# ----------------------------------------------------
try:
    upstage_embeddings
except Exception as e:
    print(f"Error loading embedding model. Do you have a GPU runtime? {e}")
    # (GPU 런타임이 아닐 경우 CPU로 fallback)
    # model_kwargs = {'device': 'cpu'}
    # embedding_model = HuggingFaceEmbeddings(
    #     model_name=model_name,
    #     model_kwargs=model_kwargs,
    #     encode_kwargs=encode_kwargs
    # )
print("Embedding model loaded.")

# ----------------------------------------------------
# 3. JSONL 로드 및 '안전 장치' 청킹
# ----------------------------------------------------
print(f"Loading '{JSONL_FILE}' and applying safety net chunking...")

# '안전 장치' (H2 섹션이 너무 클 경우를 대비한) 텍스트 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

all_final_chunks = [] # 최종적으로 FAISS에 들어갈 Document 객체 리스트

try:
    with jsonlines.open(JSONL_FILE, 'r') as reader:
        for entry in reader:
            # (1) 기본 메타데이터 (페이지 레벨)
            base_metadata = {
                "source": entry.get("source_url", "N/A"),
                "title": entry.get("title", "N/A"),
                "doc_type": entry.get("doc_type", "wex"),
            }

            # (2) Semantic Chunking된 'chunk_list' 순회
            for chunk in entry.get("chunk_list", []):
                section_text = chunk.get("text")
                section_title = chunk.get("section_title", "N/A")

                if not section_text:
                    continue

                # (3) H2 섹션 텍스트가 CHUNK_SIZE(1000자)를 넘을 경우,
                #     text_splitter가 이 텍스트를 더 작은 '미니 청크'로 자름
                split_texts = text_splitter.split_text(section_text)

                # (4) 이 '미니 청크'들을 Document 객체로 변환
                for text_piece in split_texts:
                    # 메타데이터에 'section' 정보를 추가
                    final_metadata = base_metadata.copy()
                    final_metadata["section"] = section_title

                    new_doc = Document(page_content=text_piece, metadata=final_metadata)
                    all_final_chunks.append(new_doc)

except FileNotFoundError:
    print(f"Error: '{JSONL_FILE}' not found. Please run the scraping script first.")
    sys.exit()

print(f"Total 'mini-chunks' to be indexed: {len(all_final_chunks)}")

# ----------------------------------------------------
# 4. FAISS 임베딩 및 저장
# ----------------------------------------------------
if all_final_chunks:
    print("Starting FAISS index creation... (This may take a long time)")
    start_time = time.time()

    # FAISS.from_documents()를 사용하면
    # 텍스트 청크는 임베딩되고, 메타데이터는 그대로 벡터 스토어에 저장됩니다.
    db_psychology = FAISS.from_documents(all_final_chunks, upstage_embeddings)

    end_time = time.time()
    print(f"FAISS index created successfully in {end_time - start_time:.2f} seconds.")

    # 생성된 인덱스를 파일로 저장
    db_psychology.save_local(INDEX_NAME)

    print(f"FAISS index saved to folder: '{INDEX_NAME}'")
else:
    print("No chunks were created. FAISS index not built.")

# 11/23 판례 사이트 정보 추가